In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\Constipation_Processed_Files
data = pd.read_csv('ConstipationCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\Constipation_Processed_Files


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Constipate,cg13609937,cg13847987,cg13822231,cg20728472,...,cg06819251,cg22158158,cg20161227,cg05701765,cg25437180,cg06002203,cg21722170,cg13142750,cg12743359,cg04293703
0,200991620021_R01C01,3001,2,1,1,0,0.0201,0.922355,0.011956,0.02195,...,0.900701,0.906393,0.948309,0.847871,0.647087,0.836282,0.731078,0.62803,0.883242,0.534427


In [3]:
X = data.drop(['PATNO','Sentrix','HYS','Gender','MDSP_Constipate'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7065217391304348

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100


BorutaPy finished running.

Iteration: 	35 / 100
Confirmed: 	3
Tentative: 	0
Rejected: 	20749


BorutaPy(estimator=RandomForestClassifier(n_estimators=24,
                                          random_state=RandomState(MT19937) at 0x1AD9BEA0C40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1AD9BEA0C40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  3


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head()

,CpG,Ranking
0,APPRDX,1
3546,cg27281389,1
3460,cg12298429,1
6170,cg23400985,2
17889,cg12683454,2


In [10]:
Top50.to_csv ('Constipation_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Constipation_Boruta_All.csv', index = False)